In [12]:
import MDAnalysis as mda
import numpy as np
import os
from utils import *
import warnings 

# Suppress warnings specific to MDAnalysis
warnings.filterwarnings("ignore", category=UserWarning, module="MDAnalysis")

In [13]:
# dictionary of charged protein residues with key being the Amber resname and the value being the net charge of that residue
residue_charge_dict = {'ARG':1,'LYS':1,'ASP':-1,'GLU':-1,'INI':-5}

In [14]:
# read in the system that you are about to model
complex = mda.Universe('complexes/6/qm_complex.pdb')
ini = complex.select_atoms("resname INI")

In [15]:
aka_atom_dict = get_substrate_aka_indexes(ini)
C2_index = aka_atom_dict['C2']
C2_id = ini.atoms[C2_index].index
C2_atom = complex.select_atoms("index " +  str(C2_id))

In [16]:
def simplify_integer_list(int_list):
    if not int_list:
        return ""

    # Sort the list to make sure the integers are in ascending order
    sorted_list = sorted(int_list)
    ranges = []
    start = sorted_list[0]
    end = start

    for number in sorted_list[1:]:
        if number == end + 1:
            end = number
        else:
            if start == end:
                ranges.append(f"{start}")
            else:
                ranges.append(f"{start}:{end}")
            start = number
            end = start

    # Add the last range or number
    if start == end:
        ranges.append(f"{start}")
    else:
        ranges.append(f"{start}:{end}")

    return ' '.join(ranges)

In [17]:
def get_atoms_by_distance(mol_universe:mda.Universe,dist1:float,dist2:float,atom_id:int):
    
    # get atoms a certain distance away from the carbonyl carbon  
    QM_shell= mol_universe.select_atoms("around " + str(dist1) + " index " + str(atom_id))
    # get the residues those atoms belong to
    QM_residues = set([atom.residue for atom in QM_shell])
    # do the same for the active atoms (which should include the QM atoms) 
    active_atoms = mol_universe.select_atoms("around " + str(dist2) + " index " + str(atom_id))
    active_residues = set([atom.residue for atom in active_atoms])
    
    # get all the atoms that belong to the residues for each of these groups 
    QM_atoms = mol_universe.select_atoms(" or ".join([f"resid {residue.resid}" for residue in QM_residues]))
    QM_atoms_indexes = [curr_atom.index for curr_atom in QM_atoms]
    active_atoms = mol_universe.select_atoms(" or ".join([f"resid {residue.resid}" for residue in active_residues]))
    active_atoms_indexes = [curr_atom.index for curr_atom in active_atoms]
    # find the fixed atoms 
    fixed_atoms = mol_universe.select_atoms(f"around 2.5 index {' '.join(map(str, active_atoms_indexes))}")
    fixed_atoms_indexes = [curr_atom.index for curr_atom in fixed_atoms]
    
    return QM_residues, active_residues, QM_atoms_indexes,active_atoms_indexes,fixed_atoms_indexes



In [18]:
QM_residues, active_residues, QM_atoms_indexes,active_atoms_indexes,fixed_atoms_indexes = get_atoms_by_distance(complex,6,6,C2_id)
total_charge = 0
# get charge of QM region 
for residue in QM_residues:
    resname = residue.resname
    if resname in residue_charge_dict:
        total_charge += residue_charge_dict[resname]

# simplify lists to write to file
QM_list = simplify_integer_list(QM_atoms_indexes)
active_list = simplify_integer_list(active_atoms_indexes)
fixed_list = simplify_integer_list(fixed_atoms_indexes)

In [19]:
# Define the input and output file paths
input_file = "template_QMMM_script.inp"
output_file = "complexes/6/opt_copy.inp"

# Open the input file and read its contents
with open(input_file, 'r') as file:
    content = file.read()

# Replace the {} placeholders with the variable values
content = content.replace("{}", "{" + QM_list + "}" , 1)  # First occurrence
content = content.replace("{}", "{" + active_list + "}", 1)  # Second occurrence
content = content.replace("{}", "{" + fixed_list + "}", 1)  # Third occurrence

# Add the custom line to the end
custom_line = "*pdbfile " + str(total_charge) +" 1 qm_complex.pdb\n"
content += custom_line

# Write the modified content to the output file
with open(output_file, 'w') as file:
    file.write(content)

print(f"File processed and saved as {output_file}")


File processed and saved as complexes/6/opt_copy.inp


In [20]:
def get_atoms_by_reslist(mol_universe:mda.Universe,residue_list:list):

    # get all the atoms that belong to the residues for each of these groups 
    selected_atoms = mol_universe.select_atoms(" or ".join([f"resid {residue}" for residue in residue_list]))
    selected_atoms_indexes = [curr_atom.index for curr_atom in selected_atoms]

    return selected_atoms_indexes
    




In [31]:
active_residues_resids = [residue.resid for residue in active_residues]

In [36]:
# read in the system that you are about to model
receptor = mda.Universe('receptor/qm_complex.pdb')
active_atoms_in_receptor = get_atoms_by_reslist(receptor,active_residues_resids)

QM_atoms_in_receptor = get_atoms_by_reslist(receptor,[1113,1114])
# simplify lists to write to file
receptor_QM_atoms_list = simplify_integer_list(QM_atoms_in_receptor)
receptor_active_atoms_list = simplify_integer_list(active_atoms_in_receptor)

In [37]:
# Define the input and output file paths
input_file = "template_QMMM_script.inp"
output_file = "receptor/opt.inp"

# Open the input file and read its contents
with open(input_file, 'r') as file:
    content = file.read()

# Replace the {} placeholders with the variable values
content = content.replace("{}", "{" + receptor_QM_atoms_list + "}" , 1)  # First occurrence
content = content.replace("{}", "{" + receptor_active_atoms_list + "}", 1)  # Second occurrence

# Add the custom line to the end
custom_line = "*pdbfile -3 1 qm_complex.pdb\n"
content += custom_line

# Write the modified content to the output file
with open(output_file, 'w') as file:
    file.write(content)

print(f"File processed and saved as {output_file}")


File processed and saved as receptor/opt.inp
